In [2]:
%reload_ext autoreload
%autoreload 2
    
from tqdm import tqdm

import torch
import torch.nn as nn
from matplotlib import pyplot as plt

import tasks

In [ ]:
# Hyperparameters
B = 100 # Batch size
D = 10 # Input dimensions
N = 2 # Number of recurrent neurons
T = 20 # Time steps per trial
sigma = 0.1  # Noise amplitude
learning_rate = 1e-3
num_epochs = 400
num_contexts = 2

task = tasks.VarGoNogo(D, T, sigma)
start_integration = task.decision_start_time()

# Create network
net = models.SpikingRNN(task.input_dimensionality(), N, 2, synaptic=True)

# Loss function and optimizer
def loss_function(outputs, labels, spks, alpha=1.0):
    target_loss = nn.CrossEntropyLoss()(outputs, labels)
    spike_loss = 0.1 * torch.sum(spks, dim=2)
    return target_loss + alpha * torch.mean(spike_loss)
   
# criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-2)


save_accuracy = torch.zeros(num_epochs)

# Training loop
for epoch in range(num_epochs):
    X, labels, params = task.generate_data(B, T)

    optimizer.zero_grad()
    outputs, spks, syns, mems = net(X, start_integration=start_integration)
    loss = loss_function(outputs, labels, spks, alpha=1.0)
    # loss = criterion(outputs, labels)  # Use only the last time step for loss
    loss.backward()
    optimizer.step()

    choice = torch.argmax(outputs, dim=1)
    accuracy = torch.sum(choice == labels).item() / B * 100
    save_accuracy[epoch] = accuracy
    
    if (epoch + 1) % (num_epochs // 10) == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')

X, labels, params = task.generate_data(B, T)
outputs, spks, syns, mems = net(X, start_integration=start_integration)
choice = torch.argmax(outputs, dim=1)
print(f'Failures: Choice - Labels - EmpiricalS')
errors = torch.stack((choice, labels), dim=1)[choice != labels]
print(errors)
print(f'Accuracy: {torch.sum(choice == labels).item() / B * 100:.2f}%')